In [14]:
from scipy import stats
import numpy as np

   The number of SNPs that affect a CpG site is 796,967.  
The number of SNPs that affect a CpG site that also occurs within an SV is 8,795  
The total number of SNPs identified between Brahman and Angus autosomes is 15,477,763  
The length of the Brahman autosomes is: 2,478,073,158  
The length of the Angus autosomes is: 2,468,157,877  

In [6]:
cpg_snps = 796967
cpg_snps_sv = 8795
total_snps = 15477763
brahman_len = 2478073158
angus_len = 2468157877

# The proportion of CpG SNPs in the autosomes that aren't introduced by SVs is:
print((cpg_snps - cpg_snps_sv) / total_snps)

# Compared to the proportion of SNPs in the autosomes:
print(total_snps/brahman_len)

# Enrichment of SNPs around CpG sites
print(0.05 / 0.006)

0.050922862690170406
0.0062458862241548075
8.333333333333334


In [8]:
# Is this enrichment significant?
stats.binomtest(k=(cpg_snps - cpg_snps_sv),
                n=total_snps,
                p=(total_snps / brahman_len),
                alternative='greater')

BinomTestResult(k=788172, n=15477763, alternative='greater', statistic=0.050922862690170406, pvalue=0.0)

The length of SVs identified between Brahman and Angus is 27,470,399   
The length of the non-SVs identified between Brahman and Angus is 2,450,603,277  
The number of CpGs that occur within SVs is 322,746.  
The number of CpGs that occur in non-SVs is 25,476,532  

In [11]:
cpg_sv = 322726
non_sv_cpg = 25476532
sv_len = 27470399
not_sv_len = 2450603277
# Fold enrichment of CpGs in SVs
print((cpg_sv / sv_len) / (non_sv_cpg / not_sv_len))

# Is this enrichment significant?
stats.binomtest(k=cpg_sv, n=sv_len, p=(non_sv_cpg / not_sv_len), alternative='greater')

1.130060483065982


BinomTestResult(k=322726, n=27470399, alternative='greater', statistic=0.011748136603330734, pvalue=0.0)

### Divergence between Brahman and Angus
Using the SNPs and SVs to measure divergence between Brahman and Angus, we use:  

Number of SNPs = 15477763  
Length of SVs = 27470399  
Number of SNPs that overlap with an SV = 8795  
The length of the Brahman autosomes = 2478073158
Total number of CpGs in Brahman = 25799915

((snp.shape[0] + 27000000) - len(total_snp_sv_ol)) / sum(autos_len)

In [17]:
print(f'Genome-wide mutation rate: {np.around(((15477763 + 27470399) - 8795) / 2478073158, decimals=3)}')
print('Compared to CpG mutation rate')
print(f'CpG mutation rate: {np.around(((cpg_snps + cpg_sv) - cpg_snps_sv)/25799915, decimals=3)}')

Genome-wide mutation rate: 0.017
Compared to CpG mutation rate
CpG mutation rate: 0.043
